In [0]:
%sql
MERGE INTO investments.fred.silver_metadata AS target
USING (
    SELECT 
        series_id,
        friendly_name,
        title,
        frequency,
        frequency_short,
        units,
        units_short,
        seasonal_adjustment,
        seasonal_adjustment_short,
        TO_DATE(observation_start) AS observation_start,
        TO_DATE(observation_end) AS observation_end,
        try_to_timestamp(last_updated, 'yyyy-MM-dd HH:mm:ssXXX') AS last_updated,
        CAST(popularity AS INT) AS popularity,
        notes,
        try_to_timestamp(run_timestamp, 'yyyy-MM-dd HH:mm:ss') AS run_timestamp,
        current_timestamp() AS updated_at
    FROM (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY series_id ORDER BY run_timestamp DESC) AS rn
        FROM investments.fred.bronze_metadata
    )
    WHERE rn = 1
) AS source
ON target.series_id = source.series_id
WHEN MATCHED AND (
        target.friendly_name != source.friendly_name OR
        target.title != source.title OR
        target.frequency != source.frequency OR
        target.frequency_short != source.frequency_short OR
        target.units != source.units OR
        target.units_short != source.units_short OR
        target.seasonal_adjustment != source.seasonal_adjustment OR
        target.seasonal_adjustment_short != source.seasonal_adjustment_short OR
        target.observation_start != source.observation_start OR
        target.observation_end != source.observation_end 
    )
THEN
    UPDATE SET
        target.friendly_name = source.friendly_name,
        target.title = source.title,
        target.frequency = source.frequency,
        target.frequency_short = source.frequency_short,
        target.units = source.units,
        target.units_short = source.units_short,
        target.seasonal_adjustment = source.seasonal_adjustment,
        target.seasonal_adjustment_short = source.seasonal_adjustment_short,
        target.observation_start = source.observation_start,
        target.observation_end = source.observation_end,
        target.last_updated = source.last_updated,
        target.popularity = source.popularity,
        target.notes = source.notes,
        target.run_timestamp = source.run_timestamp,
        target.updated_at = source.updated_at
WHEN NOT MATCHED THEN
    INSERT (series_id, friendly_name, title, frequency, frequency_short, units, units_short, 
            seasonal_adjustment, seasonal_adjustment_short, observation_start, observation_end, 
            last_updated, popularity, notes, run_timestamp, updated_at)
    VALUES (source.series_id, source.friendly_name, source.title, source.frequency, source.frequency_short, 
            source.units, source.units_short, source.seasonal_adjustment, source.seasonal_adjustment_short, 
            source.observation_start, source.observation_end, source.last_updated, source.popularity, 
            source.notes, source.run_timestamp, source.updated_at);

In [0]:
%sql
-- -----------------------------------------------------------------------------
-- Load Silver Rates
-- Converts data types and deduplicates by series_id + date (max run_timestamp)
-- -----------------------------------------------------------------------------
MERGE INTO investments.fred.silver_rates AS target
USING (
    SELECT 
        series_id,
        series_name,
        TO_DATE(date) AS date,
        CAST(value AS DOUBLE) AS value,
        TO_TIMESTAMP(run_timestamp, 'yyyy-MM-dd HH:mm:ss') AS run_timestamp,
        current_timestamp() AS updated_at
    FROM (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY series_id, date ORDER BY run_timestamp DESC) AS rn
        FROM investments.fred.bronze_rates
        WHERE value IS NOT NULL
    )
    WHERE rn = 1
) AS source
ON target.series_id = source.series_id AND target.date = source.date
WHEN MATCHED AND target.value != source.value THEN
    UPDATE SET
        target.series_name = source.series_name,
        target.value = source.value,
        target.run_timestamp = source.run_timestamp,
        target.updated_at = source.updated_at
WHEN NOT MATCHED THEN
    INSERT (series_id, series_name, date, value, run_timestamp, updated_at)
    VALUES (source.series_id, source.series_name, source.date, source.value, source.run_timestamp, source.updated_at);